## Airplane Crash Data Analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("Setup complete. Ready for Aviation Crash Data Analysis.")

Setup complete. Ready for Aviation Crash Data Analysis.
